# Introduction to Cryptography for Web3 Engineers

## Notebook Kernel Setup

The below enables optional features we want for speed & reuse of build artifacts, should be run at kernel startup, and all dependencies that we need to build _before_ anything else in this notebook will work.

Instead of rebuilding see the `Reset, keeping build artifacts` section to clear REPL state only. 


In [ ]:
:sccache 1
:linker lld

:dep sp-core = { version = "6.0.0", git = "https://github.com/paritytech/substrate.git", branch = "polkadot-v0.9.25" }

// Loading & building dependencies crates here takes *a while*! 
// Run this while you move on to the readings below.
// NOTE: A kernel restart removes all target artifacts except those in sccache!
// ONLY restart only if explicitly needed.

## Digital Signatures

Here we demonstrate a few parts of the [Substrate Primitives (`sp_core`)](https://paritytech.github.io/substrate/master/sp_core/index.html) library's component for interacting with keys and signatures.

In [ ]:
use sp_core::{
	blake2_256,
	crypto::Derive,
	DeriveJunction,
	Pair as _,
	sr25519::{Pair, Public},
};

## Key generation

In [ ]:
// Generate a secret key.
let (pair, mnemonic, raw_seed) = Pair::generate_with_phrase(None);
println!("Secret Phrase: {}", mnemonic);
println!("Secret Key: {:?}", raw_seed);

In [ ]:
// Derive the public key.
let pk = pair.public();
println!("Public Key: {:?}\n", pk.0);

## Signatures

In [ ]:
// Sign a message.
let message = b"Welcome to Polkadot Blockchain Academy 2022";
let signature = pair.sign(&message[..]);
println!("Message: {:?}", std::str::from_utf8(&message[..]).unwrap());
println!("Signature: {:?}", &signature);

In [ ]:
// Verify the signature.
assert!(Pair::verify(&signature, &message[..], &pk));
println!("Signature verified!\n");

In [ ]:
// Alter the message.
let tampered = b"Welcome to Polkadot Blockchain Academy 2021";
assert!(!Pair::verify(&signature, &tampered[..], &pk));
println!("Tampered Message: {:?}", std::str::from_utf8(&tampered[..]).unwrap());
println!("Signature rejected!\n");

In [ ]:
/* Message Hash */
let long_message =
	b"Welcome to Polkadot Blockchain Academy 2022. We are staying in Cambridge, which I was told is far superior to Oxford, but I should probably leave that to others to hash out.";
let message_hash = blake2_256(&long_message[..]);
let signature_on_hash = pair.sign(&message_hash[..]);
println!("Longer Message: {:?}", std::str::from_utf8(&long_message[..]).unwrap());
println!("Long Message Hash: {:?}", message_hash);

In [ ]:
// Verify the signature.
assert!(Pair::verify(&signature_on_hash, blake2_256(&long_message[..]), &pk));
println!("Signature verified!\n");

## Hard Derivation

In [ ]:
// Derive new key pairs using //polkadot and //kusama.
let pair_polkadot = Pair::from_string(&format!("{}//polkadot", &mnemonic), None);
let pk_polkadot = pair_polkadot.unwrap().public();
let pair_kusama = Pair::from_string(&format!("{}//kusama", &mnemonic), None);
let pk_kusama = pair_kusama.unwrap().public();
println!("Polkadot Public Key: {:?}", &pk_polkadot.0);
println!("Kusama Public Key: {:?}\n", pk_kusama.0);

## Soft Derivation

In [ ]:
// Derive a soft path on the Polkadot key.
let pair_polkadot_zero = Pair::from_string(&format!("{}//polkadot/0", &mnemonic), None);
let pubkey_soft_derived_with_secret = pair_polkadot_zero.unwrap().public();
println!(
	"Polkadot Soft-Derived Public Key (from secret): {:?}",
	&pubkey_soft_derived_with_secret.0
);

// Derive a soft path on the Polkadot key, but only use the _public_ material.
let pk_polkadot: Public = Public(pk_polkadot.0);
let path = vec![DeriveJunction::soft(0u8)];
let pubkey_soft_derived_without_secret = pk_polkadot.derive(path.into_iter());
println!(
	"Polkadot Soft-Derived Public Key (from pubkey): {:?}",
	&pubkey_soft_derived_without_secret.unwrap().0
);

assert_eq!(pubkey_soft_derived_with_secret, pubkey_soft_derived_without_secret.unwrap());
println!("They are equal!");

### Reset, keeping build artifacts

Run the below cell to "reset" the kernel

In [ ]:
:clear // Clear all state, **keeping compilation cache**, use this over a kernel restart when possible. You will need to re-run the :deps to have them loaded into state. 
:last_compile_dir // Show where the deps we last built live, in case you want to recover these